In [5]:
import pandas as pd
df_original = pd.read_csv('movies_dataset.csv')

C:\Users\Perettib\AppData\Local\Temp\ipykernel_10352\3718307802.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv('movies_dataset.csv')


Creamos un dataset nuevo conservando solo las columnas necesarias para el análisis y desanidando aquellas que están anidadas

In [6]:
#Seleccionamos todas las columnas relevantes para llevarlas al nuevo DataFrame
columnas_relevantes = [
    'belongs_to_collection', 'budget', 'id', 'original_language', 'genres',
    'original_title', 'overview', 'popularity', 'production_companies',
    'production_countries', 'release_date', 'revenue', 'runtime',
    'status', 'title', 'vote_average', 'vote_count'
]

# Creamos un nuevo DataFrame con las columnas seleccionadas
df_relevantes = df_original[columnas_relevantes]

# Desanidamos 'belongs_to_collection'
df_collection = pd.json_normalize(df_relevantes['belongs_to_collection'].dropna().apply(eval).tolist()).add_prefix('collection_')
df_relevantes = df_relevantes.drop(columns='belongs_to_collection').join(df_collection)

# Desanidamos 'genres'
df_relevantes['genres'] = df_relevantes['genres'].apply(lambda x: eval(x) if isinstance(x, str) else x)
df_genres = df_relevantes.explode('genres')
df_genres = pd.json_normalize(df_genres['genres']).add_prefix('genre_')
df_relevantes = df_relevantes.join(df_genres)

# Desanidamos 'production_companies'
df_relevantes['production_companies'] = df_relevantes['production_companies'].apply(lambda x: eval(x) if isinstance(x, str) else x)
df_companies = df_relevantes.explode('production_companies')
df_companies = pd.json_normalize(df_companies['production_companies']).add_prefix('company_')
df_relevantes = df_relevantes.join(df_companies)

# Desanidamos 'production_countries'
df_relevantes['production_countries'] = df_relevantes['production_countries'].apply(lambda x: eval(x) if isinstance(x, str) else x)
df_countries = df_relevantes.explode('production_countries')
df_countries = pd.json_normalize(df_countries['production_countries']).add_prefix('country_')
df_relevantes = df_relevantes.join(df_countries)


# Guardamos el nuevo DataFrame en un archivo CSV
df_relevantes.to_csv('dataset_movies_desanidado.csv', index=False)

print("El nuevo dataset fue creado y guardado como 'dataset_movies_desanidado.csv'.")


El nuevo dataset fue creado y guardado como 'dataset_movies_desanidado.csv'.


Importamos el nuevo dataset generado y visualizar las columnas desanidadas

In [8]:
df = pd.read_csv("dataset_movies_desanidado.csv")
df.head()


C:\Users\Perettib\AppData\Local\Temp\ipykernel_10352\496703026.py:1: DtypeWarning: Columns (6,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset_movies_desanidado.csv")


,budget,id,original_language,genres,original_title,overview,popularity,production_companies,production_countries,release_date,...,collection_id,collection_name,collection_poster_path,collection_backdrop_path,genre_id,genre_name,company_name,company_id,country_iso_3166_1,country_name
0,30000000,862,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America
1,65000000,8844,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,35.0,Comedy,TriStar Pictures,559.0,US,United States of America
2,0,15602,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,10751.0,Family,Teitler Film,2550.0,US,United States of America
3,16000000,31357,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,645.0,James Bond Collection,/HORpg5CSkmeQlAolx3bKMrKgfi.jpg,/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg,12.0,Adventure,Interscope Communications,10201.0,US,United States of America
4,0,11862,en,"[{'id': 35, 'name': 'Comedy'}]",Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,...,117693.0,Balto Collection,/w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg,/9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg,14.0,Fantasy,Warner Bros.,6194.0,US,United States of America


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   budget                    45466 non-null  object 
 1   id                        45466 non-null  object 
 2   original_language         45455 non-null  object 
 3   genres                    45466 non-null  object 
 4   original_title            45466 non-null  object 
 5   overview                  44512 non-null  object 
 6   popularity                45461 non-null  object 
 7   production_companies      45463 non-null  object 
 8   production_countries      45463 non-null  object 
 9   release_date              45379 non-null  object 
 10  revenue                   45460 non-null  float64
 11  runtime                   45203 non-null  float64
 12  status                    45379 non-null  object 
 13  title                     45460 non-null  object 
 14  vote_a

Eliminamos las columnas video,imdb_id,adult,original_title,poster_path y homepage

In [10]:
columns_to_drop = ["collection_poster_path", "collection_backdrop_path", "genres", "production_companies", "production_countries", "country_iso_3166_1"]
df = df.drop(columns=columns_to_drop, errors='ignore')


Las columnas 'collection_id' y 'collection_name' tienen muchos nulos y no son necesarias para el sistema de recomendación.

No es necesario tener la columna del tiempo que dura la pelicula en este caso no la vamos a utilizar a la columna

In [11]:
columns_to_drop = ["collection_id", "collection_name", "runtime"]
df = df.drop(columns=columns_to_drop, errors='ignore')

La columna original_language: no es relevante para la creación de la api ni del modelo


Las columnas genre_id y genre_name: en este caso no usamos géneros en la api, es innecesaria.


Las columnas company_id, company_name : no tenemos que realizar ninguna función relacionada con las productoras.

La columna country_name: no necesitamos los nombre de paises donde se produjo la película 


In [12]:
columns_to_drop = ["original_language", "genre_id", "genre_name", "company_id", "company_name", "country_name"]
df = df.drop(columns=columns_to_drop, errors='ignore')

In [13]:
df

,budget,id,original_title,overview,popularity,release_date,revenue,status,title,vote_average,vote_count
0,30000000,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,Released,Toy Story,7.7,5415.0
1,65000000,8844,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,Released,Jumanji,6.9,2413.0
2,0,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,Released,Grumpier Old Men,6.5,92.0
3,16000000,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,Released,Waiting to Exhale,6.1,34.0
4,0,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,Released,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...
45461,0,439050,رگ خواب,Rising and falling between a man and woman.,0.072051,NaN,0.0,Released,Subdue,4.0,1.0
45462,0,111109,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,Released,Century of Birthing,9.0,3.0
45463,0,67758,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,Released,Betrayal,3.8,6.0
45464,0,227506,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,Released,Satan Triumphant,0.0,0.0


Rellenamos valores NaN con 0 (si es apropiado) y Convertir la columna a entero

In [14]:
df['vote_count'] = df['vote_count'].fillna(0)
df['vote_count'] = df['vote_count'].astype(int)

In [ ]:
df.head()

,budget,id,original_title,overview,popularity,release_date,revenue,status,title,vote_average,vote_count
0,30000000,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,Released,Toy Story,7.7,5415
1,65000000,8844,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,Released,Jumanji,6.9,2413
2,0,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,Released,Grumpier Old Men,6.5,92
3,16000000,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,Released,Waiting to Exhale,6.1,34
4,0,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,Released,Father of the Bride Part II,5.7,173
...,...,...,...,...,...,...,...,...,...,...,...
45461,0,439050,رگ خواب,Rising and falling between a man and woman.,0.072051,NaN,0.0,Released,Subdue,4.0,1
45462,0,111109,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,Released,Century of Birthing,9.0,3
45463,0,67758,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,Released,Betrayal,3.8,6
45464,0,227506,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,Released,Satan Triumphant,0.0,0


Rellenamos valores nulos en 'revenue' y 'budget' con 0

In [16]:
df['revenue'].fillna(0, inplace=True)
df['budget'].fillna(0, inplace=True)

C:\Users\Perettib\AppData\Local\Temp\ipykernel_10352\2153194234.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['revenue'].fillna(0, inplace=True)
C:\Users\Perettib\AppData\Local\Temp\ipykernel_10352\2153194234.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

Verificamos que se haya agregado los 0 por los null 

In [17]:
columnas_transformadas = df[['revenue', 'budget']]
print(columnas_transformadas)

           revenue    budget
0      373554033.0  30000000
1      262797249.0  65000000
2              0.0         0
3       81452156.0  16000000
4       76578911.0         0
...            ...       ...
45461          0.0         0
45462          0.0         0
45463          0.0         0
45464          0.0         0
45465          0.0         0

[45466 rows x 2 columns]


Visualizamos la cantidad de nulos del dataset

In [18]:
null_counts = df.isnull().sum()
print(null_counts)

budget              0
id                  0
original_title      0
overview          954
popularity          5
release_date       87
revenue             0
status             87
title               6
vote_average        6
vote_count          0
dtype: int64


In [19]:
df.head()

,budget,id,original_title,overview,popularity,release_date,revenue,status,title,vote_average,vote_count
0,30000000,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,Released,Toy Story,7.7,5415
1,65000000,8844,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,Released,Jumanji,6.9,2413
2,0,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,Released,Grumpier Old Men,6.5,92
3,16000000,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,Released,Waiting to Exhale,6.1,34
4,0,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,Released,Father of the Bride Part II,5.7,173


Verificamos el tipo de dato de la columna release_date 

In [20]:
tipo_dato = df['release_date'].dtype
print(tipo_dato)

object


Cambiamos el tipo de dato a datetime

In [21]:
try:
    df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
except Exception as e:
    print(f"Error al convertir 'release_date': {e}")


In [22]:
tipo_dato = df['release_date'].dtype
print(tipo_dato)

datetime64[ns]


Verificamos si efectivamente cambiaron los datos y se adptaron al formato AAAA-mm-dd

In [23]:
columna_transformada = df[['release_date']]
print(columna_transformada)

      release_date
0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
...            ...
45461          NaT
45462   2011-11-17
45463   2003-08-01
45464   1917-10-21
45465   2017-06-09

[45466 rows x 1 columns]


Eliminamos los nulos de la columna release_date

In [24]:
df = df.dropna(subset=['release_date'])

Verificamos si hay nulos

In [25]:
print(df['release_date'].isna().sum())

0


In [26]:
print(df['release_date'].isnull().sum())

0


Creamos la columna 'release_year' extrayendo el año de 'release_date'

In [27]:
df['release_year'] = df['release_date'].dt.year

# Reemplazamos los valores nulos con un valor 0 
df['release_year'] = df['release_year'].fillna(0).astype(int)

C:\Users\Perettib\AppData\Local\Temp\ipykernel_10352\1110542261.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_year'] = df['release_date'].dt.year
C:\Users\Perettib\AppData\Local\Temp\ipykernel_10352\1110542261.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_year'] = df['release_year'].fillna(0).astype(int)


Verificamos que la columna se haya creado correctamente y que contenga únicamente los datos correspondientes al año de la fecha de estreno

In [28]:
print(df[['release_date', 'release_year']].head())

  release_date  release_year
0   1995-10-30          1995
1   1995-12-15          1995
2   1995-12-22          1995
3   1995-12-22          1995
4   1995-02-10          1995


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   budget          45376 non-null  object        
 1   id              45376 non-null  object        
 2   original_title  45376 non-null  object        
 3   overview        44435 non-null  object        
 4   popularity      45376 non-null  object        
 5   release_date    45376 non-null  datetime64[ns]
 6   revenue         45376 non-null  float64       
 7   status          45296 non-null  object        
 8   title           45376 non-null  object        
 9   vote_average    45376 non-null  float64       
 10  vote_count      45376 non-null  int32         
 11  release_year    45376 non-null  int32         
dtypes: datetime64[ns](1), float64(2), int32(2), object(7)
memory usage: 4.2+ MB


Convertimos las  columnas budget y revenue a a entero

In [30]:
# Convertimos las columnas a numérico, forzando los errores a NaN
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

# Reemplazamos los valores NaN con 0
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

# Convertimos las columnas a enteros
df['revenue'] = df['revenue'].astype(int)
df['budget'] = df['budget'].astype(int)

C:\Users\Perettib\AppData\Local\Temp\ipykernel_10352\975643497.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
C:\Users\Perettib\AppData\Local\Temp\ipykernel_10352\975643497.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
C:\Users\Perettib\AppData\Local\Temp\ipykernel_10352\975643497.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

Creamos la columna 'return' calculando revenue / budget, con 0 cuando falten datos

Reemplazamos valores NaN (resultado de la división por 0 o datos faltantes) con 0


In [31]:
df['return'] = df['revenue'] / df['budget']
df['return'].fillna(0, inplace=True)

C:\Users\Perettib\AppData\Local\Temp\ipykernel_10352\803909767.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['return'] = df['revenue'] / df['budget']
C:\Users\Perettib\AppData\Local\Temp\ipykernel_10352\803909767.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['return'].f

Verificamos que se haya creado con exito 

In [32]:
print(df[["budget","revenue", "return" ]])

         budget    revenue     return
0      30000000  373554033  12.451801
1      65000000  262797249   4.043035
2             0          0   0.000000
3      16000000   81452156   5.090760
4             0   76578911        inf
...         ...        ...        ...
45460         0          0   0.000000
45462         0          0   0.000000
45463         0          0   0.000000
45464         0          0   0.000000
45465         0          0   0.000000

[45376 rows x 3 columns]


Reemplazamos los valores 'inf' en la columna 'return' con 0

In [33]:
import numpy as np
df['return'].replace([np.inf, -np.inf], 0, inplace=True)

C:\Users\Perettib\AppData\Local\Temp\ipykernel_10352\264360229.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['return'].replace([np.inf, -np.inf], 0, inplace=True)
C:\Users\Perettib\AppData\Local\Temp\ipykernel_10352\264360229.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['return'].replace([np.inf, -np.inf], 0, inplace=True)


Verificamos las 3 columnas (budget, revenue, return)

In [37]:
print(df[["budget","revenue", "return" ]])

         budget    revenue     return
0      30000000  373554033  12.451801
1      65000000  262797249   4.043035
2             0          0   0.000000
3      16000000   81452156   5.090760
4             0   76578911   0.000000
...         ...        ...        ...
45460         0          0   0.000000
45462         0          0   0.000000
45463         0          0   0.000000
45464         0          0   0.000000
45465         0          0   0.000000

[45376 rows x 3 columns]


In [ ]:
df.head()

,budget,id,original_title,overview,popularity,release_date,revenue,status,title,vote_average,vote_count,release_year,return
0,30000000,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,Released,Toy Story,7.7,5415,1995,12.451801
1,65000000,8844,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,Released,Jumanji,6.9,2413,1995,4.043035
2,0,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0,Released,Grumpier Old Men,6.5,92,1995,0.000000
3,16000000,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156,Released,Waiting to Exhale,6.1,34,1995,5.090760
4,0,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911,Released,Father of the Bride Part II,5.7,173,1995,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,0,30840,Robin Hood,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0,Released,Robin Hood,5.7,26,1991,0.000000
45462,0,111109,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,2011-11-17,0,Released,Century of Birthing,9.0,3,2011,0.000000
45463,0,67758,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0,Released,Betrayal,3.8,6,2003,0.000000
45464,0,227506,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0,Released,Satan Triumphant,0.0,0,1917,0.000000


Guardamos los cambios 

In [35]:
df.to_csv('dataset_movies_desanidado.csv', index=False)

print("Los cambios se han guardado en 'dataset_movies_desanidado.csv'.")

Los cambios se han guardado en 'dataset_movies_desanidado.csv'.


## Data set credits_df

In [ ]:
credits_df = pd.read_csv('credits.csv')
credits_df

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


Desanidamos las columnas

In [ ]:
# Seleccionamos las columnas 'id', 'cast', 'crew'
columnas_relevantes = ['id', 'cast', 'crew']

# Creamos un nuevo DataFrame con las columnas seleccionadas
df_relevantes = credits_df[columnas_relevantes]

# Desanidamos 'cast'
df_relevantes['cast'] = df_relevantes['cast'].apply(lambda x: eval(x) if isinstance(x, str) else x)
df_cast = df_relevantes.explode('cast')
df_cast = pd.json_normalize(df_cast['cast']).add_prefix('cast_')
df_relevantes = df_relevantes.drop(columns='cast').join(df_cast)

# Desanidamos 'crew'
df_relevantes['crew'] = df_relevantes['crew'].apply(lambda x: eval(x) if isinstance(x, str) else x)
df_crew = df_relevantes.explode('crew')
df_crew = pd.json_normalize(df_crew['crew']).add_prefix('crew_')
df_relevantes = df_relevantes.drop(columns='crew').join(df_crew)

# Guardamos el nuevo DataFrame en un archivo CSV
df_relevantes.to_csv('dataset_credits_desanidados.csv', index=False)

print("El nuevo dataset ha sido creado y guardado como 'dataset_credits_desanidados.csv'.")


El nuevo dataset ha sido creado y guardado como 'dataset_credits_desanidados.csv'.


Importamos el nuevo dataset generado y visualizar las columnas desanidadas

In [ ]:
df1 = pd.read_csv("dataset_credits_desanidados.csv")
df1.head()


,id,cast_cast_id,cast_character,cast_credit_id,cast_gender,cast_id,cast_name,cast_order,cast_profile_path,crew_credit_id,crew_department,crew_gender,crew_id,crew_job,crew_name,crew_profile_path
0,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,8844,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,15602,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,31357,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,11862,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


Eliminamos las columnas que no necesitamos

In [ ]:
columns_to_drop = [
    'cast_cast_id', 'cast_credit_id', 'cast_gender', 'cast_id', 'cast_order', 'cast_profile_path',
    'crew_credit_id', 'crew_gender', 'crew_id', 'crew_profile_path']

df1 = df1.drop(columns=columns_to_drop, errors='ignore')


In [ ]:
df1.head()

,id,cast_character,cast_name,crew_department,crew_job,crew_name
0,862,Woody (voice),Tom Hanks,Directing,Director,John Lasseter
1,8844,Buzz Lightyear (voice),Tim Allen,Writing,Screenplay,Joss Whedon
2,15602,Mr. Potato Head (voice),Don Rickles,Writing,Screenplay,Andrew Stanton
3,31357,Slinky Dog (voice),Jim Varney,Writing,Screenplay,Joel Cohen
4,11862,Rex (voice),Wallace Shawn,Writing,Screenplay,Alec Sokolow
...,...,...,...,...,...,...
45471,439050,Mary Lambetta,Alix Koromzay,Writing,Screenplay,Joe Eszterhas
45472,111109,Todd Hackett,Todd Field,Production,Producer,Don Simpson
45473,67758,Lou,Tom Irwin,Production,Producer,Jerry Bruckheimer
45474,227506,Hugh Crain,Charles Gunning,Sound,Original Music Composer,Giorgio Moroder


Guardamos los cambios 

In [ ]:
df1.to_csv("dataset_credits_desanidados.csv", index=False)

print("Los cambios se han guardado en dataset_credits_desanidados.csv")

Los cambios se han guardado en dataset_credits_desanidados.csv


## Transformamos los csv a Parquet

In [45]:
movies_df = pd.read_csv('dataset_movies_desanidado.csv')
credits_df = pd.read_csv('dataset_credits_desanidados.csv')

# Convertimos a formato Parquet
movies_df.to_parquet('dataset_movies.parquet', engine='pyarrow', index=False)
credits_df.to_parquet('dataset_credits.parquet', engine='pyarrow', index=False)

print("Conversión completada.")

Conversión completada.
